In [102]:
from entities import *
from constants import *
from simulations import *
from tournament_roster import *
from constants import TEAMS_DATA, STAT_EFFECTS

import pickle
import numpy as np
import pandas as pd
import openpyxl as xl

from pathlib import Path

LOAD = True
PRECOMPUTED_GAMES = Path.cwd() / 'data' / 'pre_computed_games.pkl'
BLANK_BRACKET = Path.cwd() / 'resources' / 'blank_bracket.xlsx'
OUTPUT = Path.cwd() / 'output'

In [5]:
with open(TEAMS_DATA, 'rb') as f:
    teams = pickle.load(f)

with open(STAT_EFFECTS, 'rb') as f:
    stats = pickle.load(f)

if LOAD:
    with open(PRECOMPUTED_GAMES, 'rb') as f:
        games = pickle.load(f)

In [23]:
wb = xl.load_workbook('resources/blank_bracket.xlsx', data_only=True)

tournament_roster = [teams[wb['Teams'][f'C{i}'].value] for i in range(2, 66)]
tournament_roster

[<Team: Abilene Christian Record: 10-10>,
 <Team: Air Force Record: 3-17>,
 <Team: Akron Record: 17-4>,
 <Team: Alabama Record: 14-6>,
 <Team: Alabama A&M Record: 11-9>,
 <Team: Alabama State Record: 6-14>,
 <Team: Albany (NY) Record: 7-15>,
 <Team: Alcorn State Record: 3-16>,
 <Team: American Record: 13-9>,
 <Team: Appalachian State Record: 14-9>,
 <Team: Arizona Record: 21-0>,
 <Team: Arizona State Record: 11-10>,
 <Team: Arkansas Record: 16-5>,
 <Team: Arkansas State Record: 13-9>,
 <Team: Arkansas-Pine Bluff Record: 7-13>,
 <Team: Army Record: 9-13>,
 <Team: Auburn Record: 14-7>,
 <Team: Austin Peay Record: 14-6>,
 <Team: Ball State Record: 7-13>,
 <Team: Baylor Record: 11-9>,
 <Team: Bellarmine Record: 8-13>,
 <Team: Belmont Record: 19-3>,
 <Team: Bethune-Cookman Record: 9-11>,
 <Team: Binghamton Record: 5-18>,
 <Team: Boise State Record: 13-8>,
 <Team: Boston College Record: 9-11>,
 <Team: Florida Record: 15-6>,
 <Team: Duke Record: 19-1>,
 <Team: Bradley Record: 14-8>,
 <Team: B

In [ ]:
def round_of_games(team_list: list, n: int=20, summary=False):
    winners, losers = [], []
    schools = []
    records = []
    analyses = []

    for i in range(0, len(team_list), 2):
        game = simulate_n_games(team_list[i], team_list[i+1], n, summary=summary)
        winners.append(teams[game['Winner']].name)
        losers.append(teams[game['Loser']].name)
        analyses.append(game['Analysis'])
        schools.append([game['Team 1'], game['Team 2']])
        records.append([game['Win Count 1'], game['Win Count 2']])
    
    return {'Winners': winners, 
            'Losers': losers, 
            'Analysis': analyses, 
            'Schools': schools,
            'Records': records
        }

def first_round(region: dict, n: int=20):
    first_round_teams = [teams[v] for v in region.values()]
    return round_of_games(first_round_teams, n=n)

# Precalculated Games

In [99]:
def precomputed_games(team_1, team_2):
    for game in games:
        game_players = [game['Winner'], game['Loser']]
        if (team_1 in game_players) and (team_2 in game_players):
            # print(game['Winner'])
            # print(game['Record'])
            return game['Winner']

def precomputed_games_from_list(l):
    for i in range(0, len(l), 2):
        yield precomputed_games(l[i], l[i+1])

def precomputed_region(roster_list):
    first_round = list(precomputed_games_from_list(roster_list))
    second_round = list(precomputed_games_from_list(first_round))
    third_round = list(precomputed_games_from_list(second_round))
    fourth_round = list(precomputed_games_from_list(third_round))
    return first_round, second_round, third_round, fourth_round

In [100]:
for region in [south, west, east, midwest]:
    print(precomputed_region(list(region.values())))

(['Buffalo', 'Butler', 'Cal State Fullerton', 'Cal State Northridge', 'Campbell', 'Central Arkansas', 'Central Michigan', 'Charleston Southern'], ['Buffalo', 'Cal State Northridge', 'Central Arkansas', 'Charleston Southern'], ['Buffalo', 'Central Arkansas'], ['Buffalo'])
(['Austin Peay', 'Baylor', 'Bellarmine', 'Binghamton', 'Boston College', 'Duke', 'Brigham Young', 'Brown'], ['Austin Peay', 'Binghamton', 'Duke', 'Brigham Young'], ['Austin Peay', 'Duke'], ['Duke'])
(['Abilene Christian', 'Akron', 'Alabama State', 'Alcorn State', 'American', 'Arizona', 'Arkansas', 'Arkansas-Pine Bluff'], ['Akron', 'Alcorn State', 'Arizona', 'Arkansas'], ['Akron', 'Arizona'], ['Arizona'])
(['Chicago State', 'Clemson', 'Coastal Carolina', 'Colgate', 'Colorado', 'Connecticut', 'Cornell', 'Creighton'], ['Clemson', 'Colgate', 'Connecticut', 'Creighton'], ['Colgate', 'Creighton'], ['Colgate'])


# Making A Copy

In [106]:
import shutil

new_bracket = OUTPUT / 'Bracket1.xlsx'

shutil.copyfile(BLANK_BRACKET, new_bracket)

WindowsPath('c:/Users/amgal/Documents/Data Science/KingfisherCourt/output/Bracket1.xlsx')

In [ ]:
# TODO: Make a new copy of the bracket .xlsx file
# TODO: Update spreadsheet copy with this